In [ ]:
#Useful imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2
from matplotlib import rc
rc('text', usetex=True)

In [ ]:
FOLDER = 'plots/'

def full_frame(width=None, height=None):
    ''' Nearly completely remove all borders from a plot. '''
    import matplotlib as mpl
    mpl.rcParams['savefig.pad_inches'] = 0
    figsize = None if width is None else (width, height)
    fig = plt.figure(figsize=figsize)
    ax = plt.axes([0,0,1,1], frameon=False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.autoscale(tight=True)
    return fig, ax
    
def plot_matrix(A, saveas='', cmap='RdBu', vmin=None, vmax=None):
    fig, ax = full_frame(A.shape[1], A.shape[0])
    ax.matshow(A, cmap=cmap, vmin=vmin, vmax=vmax)
    
    if saveas != '':
        plt.savefig(FOLDER + saveas)

## Setup

In [ ]:
kernel = np.array([[-1, -2], [-3, -4]])
S = np.array([3, 4])
np.random.seed(1)
image = np.random.uniform(0, 1.0, size=S)
image[0, 0] = 1.0
image[-1, -1] = 0.0
plot_matrix(image, 'image.eps', 'gray')

#kernel = np.array([[-1, 1]])
plot_matrix(kernel, 'k.eps', vmax=0)

## Original 2D convolution

In [ ]:
from scipy.signal import convolve2d

result_orig = convolve2d(image, kernel, mode='valid', boundary='symm')

# make sure the min and max of matrix plots are always the same. 
vmin = np.min(result_orig)
vmax = np.max(result_orig)
plot_matrix(result_orig, 'k.eps', 'gray', vmin=vmin, vmax=vmax)

## Vectorized 2D convolution

In [ ]:
from scipy.linalg import circulant

## Setup
n = S[0]*S[1]

kernel_vect = np.zeros(S)
kernel_vect[:kernel.shape[0], :kernel.shape[1]] = kernel[::-1, ::-1]
kernel_vect.resize([1, n])
plot_matrix(kernel_vect, '', vmax=0)

# Because of the convention of scipy's circulant implementation we need to undo their flipping, thus the transpose. 
kernel_matrix = np.zeros((n, n))

kernel_matrix = circulant(kernel_vect).T
plot_matrix(kernel_matrix, 'k_matrix.eps', vmax=0)

image_vect = np.reshape(image, [-1, 1])
plot_matrix(image_vect, 'image_vect.eps', 'gray')
plot_matrix(image_vect.T, 'image_vect_t.eps', 'gray')

In [ ]:
## Convolution

result_vect_vect = kernel_matrix.dot(image_vect)
plot_matrix(result_vect_vect, 'result_vect_vect.eps', 'gray', vmin=vmin, vmax=vmax)
plot_matrix(result_vect_vect.T, 'result_vect_vect_T.eps', 'gray', vmin=vmin, vmax=vmax)

result_vect = result_vect_vect.reshape(S)
plot_matrix(result_vect, 'resul_vect.eps', 'gray', vmin=vmin, vmax=vmax)

assert np.allclose(result_vect[:2, :3], result_orig)

## Practical implementation

In [ ]:
from psf2otf import psf2otf

otf_kernel = psf2otf(kernel, S)
fft_image = np.fft.fft2(image)
fft_result_psf2otf = np.multiply(otf_kernel, fft_image)

result_psf2otf = np.fft.ifft2(fft_result_psf2otf)

plot_matrix(np.real(result_psf2otf), 'real_result_psf2otf.eps', 'gray', vmin=vmin, vmax=vmax)
assert np.allclose(result_psf2otf[:2, :3], result_orig)

In [ ]:
# Visualization of PSF2OTF

from psf2otf import zero_pad

psf = np.array([[-1, -2], [-3, -4]])
shape = np.array([3, 4])

inshape = psf.shape

# Pad the PSF to outsize
psf = zero_pad(psf, shape, position='corner')
plot_matrix(psf, 'psf2otf_1.eps')

# Circularly shift OTF so that the 'center' of the PSF is
# [0,0] element of the array
for axis, axis_size in enumerate(inshape):
    psf = np.roll(psf, -int(axis_size / 2), axis=axis)

plot_matrix(psf, 'psf2otf_2.eps')

# Compute the OTF
otf = np.fft.fft2(psf)
plot_matrix(np.real(otf))
plot_matrix(np.imag(otf))
plot_matrix(np.abs(otf), 'psf2otf_3.eps')

# Estimate the rough number of operations involved in the FFT
# and discard the PSF imaginary part if within roundoff error
# roundoff error  = machine epsilon = sys.float_info.epsilon
# or np.finfo().eps
n_ops = np.sum(psf.size * np.log2(psf.shape))
otf = np.real_if_close(otf, tol=n_ops)